<a href="https://colab.research.google.com/github/ichekhovskikh/skip-thought-vectors/blob/master/skip_thoughts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка

Установим флаг floatX=float32, чтобы избежать ошибок типа TypeError.

In [0]:
!echo -e "\n[global]\nfloatX=float32\n" >> ~/.theanorc

Необходимо подлючить Google Drive, где по пути skip-thoughts-master/training располагается библиотека для работы со Skip Throughts Vectors. Папки training располагается в репозитории https://github.com/ichekhovskikh/skip-thought-vectors

In [4]:
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Сделать импорт библиотеки

In [6]:
import importlib.util
import sys

sys.path.append('/content/drive/My Drive/skip-thoughts-master/training')
import vocab
import train
import tools

DuplicateSectionError: ignored

In [11]:
!pip install pymorphy2[fast]

import gensim
from gensim.models import Word2Vec, KeyedVectors
import os
import collections
import smart_open
import random
import json
import urllib.request
import pymorphy2
import nltk

nltk.download('stopwords')

from nltk import tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели doc2vec. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель на собственном корпусе. Этот корпус содержит 70 текстов.

И мы проверим нашу модель на глаз, используя тестовый корпус, который содержит 7 документов.

Dataset состоит из трех строк: id (идентификатор строки), text (текст статьи), tag (идентификатор самой статьи, вектор которого будем обучать)

In [0]:
train_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/data.json'
test_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/test.json'

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла, предварительно обрабатываем каждый текст датасета, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.), лемматизацию, удаление стоп слов и возвращаем список слов. Для обучения модели нам нужно будет связать тег с каждым документом учебного корпуса. В нашем случае тег - это идентификатор статьи.

In [0]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(words):
    for word in words:
        yield morph.parse(word)[0].normal_form

Удаление стоп слов:

In [0]:
russian_stopwords = stopwords.words("russian")

def remove_stopwords(words):
    return [word for word in words if word not in russian_stopwords]

Предобработка текста статьи:

In [0]:
def advanced_preprocess(text):
    normalized_text = gensim.utils.simple_preprocess(text)
    normalized_text = remove_stopwords(normalized_text)
    normalized_text = list(lemmatize(normalized_text))
    return normalized_text

Отрытие файла с корпусом статей:

In [0]:
def read_corpus(corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for article in corpus:
            sentences = tokenize.sent_tokenize(article['text'], 'russian')
            normalized_sentences = []
            for sentence in sentences:
                normalized_sentences.append(' '.join(advanced_preprocess(sentence)))
            yield normalized_sentences

Получение исходного текста статьи по индексу:

In [0]:
def get_article_text_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['text']

Загружаем корпуса:

In [0]:
train_corpus = list(read_corpus(train_path))
test_corpus = list(read_corpus(test_path))

Давайте посмотрим на учебный и тестовый корпуса:

In [21]:
print(train_corpus[:2])

[['общественный жизнь экономика рынок каждый готовить блин сковорода антипригарный покра тие знать важный температура приготовление который зависеть цвета форма изделие домаш условие некоторый колебание цвет толщина форма вполне приемлемый промышленный произа водство блин сложный процесс неоднородность готовый продукт становиться больший пра пятствие успешный реали зации торговый сеть новое технический решение стабилизация качество промышленный ный производство блин разрабо таль специалист шведский фирма formcook промышленный печь термообработка самый разно образный продукт мясо птица рыба овощ пять год поставлять россия компания агро надёжный высокопроиза водительный оборудование успеть завоевать авторитет российский специалист мясной рыбный молочный промышленность сис тема общепит инженер конструктор фирма formcook промышленный производство блин год спустя запустить эксплуата цию один инновационный система приготовление пищевой промышленность течение десяток год обходиться старомодны

In [22]:
print(test_corpus[:2])

[['язык ассемблер машинный ориентировать язык низкий уровень', 'команда прямо соответствовать отдельный команда машина также предоставлять дополнительный возможность облегчение такой макрокоманда выражение средство обеспечение модульность программа', 'рассматриваться автокод расширить конструкция язык высокий уровень', 'являться существенно платформо зависимый', 'язык ассемблер различный аппаратный платформа несовместимый хотя мочь целое подобный', 'русский язык именоваться просто ассемблер типичный выражение тип писать программа ассемблер строго говорить неверный ассемблер именоваться утилит трансляция программа язык ассемблер объектный код компьютер'], ['перитонит развиваться вследствие бактериальный инфицирование брюшной полость', 'подавлять большинство пациент перитонит обнаруживать различный сочетание аэробный анаэробный микроорганизм', 'отсутствие рост микрофлора стандартный исследование условие гнойный каловый перитонит свидетельствовать лишь высокий вероятность доминировать ана

Создайте три пустых файла в Google Drive для сохранения в них модели сети, словаря, модели Word2Vec

In [0]:
path_to_model = '/content/drive/My Drive/sample_data/model.npz',
path_to_dictionary = '/content/drive/My Drive/sample_data/vocab.pkl',
path_to_word2vec = '/content/drive/My Drive/sample_data/word2vec.bin'

# Обучение модели
## Создание словаря
Состовим словарь из тренеровочного корпуса предложений и сохраним его по пути path_to_dictionary для дальнейшего использования

Объединение всех предложений в один список

In [0]:
def flatten(text):
    return [item for sublist in text for item in sublist]

Создание и сохранение словаря

In [0]:
sentenses = flatten(train_corpus)
worddict, wordcount = vocab.build_dictionary(sentenses)
vocab.save_dictionary(worddict, wordcount, path_to_dictionary)

 ## Создание модели Word2Vec
Состовим модель Word2Vec из тренеровочного корпуса предложений и сохраним его по пути path_to_word2vec для дальнейшего использования

Разделить предложение на слова

In [0]:
def word_tokenize(text):
    flat_text = flatten(text)
    return [item.split(' ') for item in flat_text]

Создание и сохранение модели Word2Vec

In [0]:
words = word_tokenize(train_corpus)
model = Word2Vec(words, size=300, window=5, min_count=1, workers=4)
model.wv.save_word2vec_format(path_to_word2vec, binary=True)

In [0]:
text = ["первое предложение", "второе предложение", "третье предложение", "что-то еще предложение"]
worddict, wordcount = vocab.build_dictionary(text)
vocab.save_dictionary(worddict, wordcount, "/content/drive/My Drive/sample_data/vocab.pkl")

train.trainer(
    text, 
    n_words=len(worddict) + 2,
    dictionary="/content/drive/My Drive/sample_data/vocab.pkl",
    saveto="/content/drive/My Drive/sample_data/model.npz",
    saveFreq=1)



In [0]:
def cosine_similarity(first_vector, second_vector):
    numerator = sum([x * y for x, y in zip(first_vector, second_vector)])
    first_norm = pow(sum([x * x for x in first_vector]), 0.5)
    second_norm = pow(sum([x * x for x in second_vector]), 0.5)
    return numerator / (first_norm * second_norm)

In [0]:
from gensim.models import Word2Vec, KeyedVectors

text_split_on_words = [['первое', 'предложение'], ['второе', 'предложение'], ['третье', 'предложение'], ['что-то', 'еще', 'предложение']]

model = Word2Vec(text_split_on_words, size=300, window=5, min_count=1, workers=4)
model.wv.save_word2vec_format('/content/drive/My Drive/sample_data/word2vec.bin', binary=True)
embed_map = KeyedVectors.load_word2vec_format('/content/drive/My Drive/sample_data/word2vec.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model = tools.load_model(
    path_to_model = '/content/drive/My Drive/sample_data/model.npz',
    path_to_dictionary = '/content/drive/My Drive/sample_data/vocab.pkl',
    path_to_word2vec = '/content/drive/My Drive/sample_data/word2vec.bin')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Loading dictionary...
Creating inverted dictionary...
Loading model options...
Loading model parameters...
Compiling encoder...
Creating word lookup tables...
Packing up...


In [0]:
vectors = tools.encode(model, text)

In [0]:
print(cosine_similarity(vectors[0], vectors[1]))

0.45222118707126796
